In [1]:
from opcua import Client
import time

def extract_trend_node_names(endpoint, probe_node_id="ns=2;s=Local.iCIR.Probe1.Trends"):
    """
    Connects to an OPC UA server and extracts the names of child nodes under the specified Probe1 Trends node.
    
    Args:
        endpoint (str): OPC UA server endpoint (e.g., "opc.tcp://localhost:62552/iCOpcUaServer")
        probe_node_id (str): NodeId of the Trends node for Probe1
    
    Returns:
        List of trend node display names (str)
    """
    client = Client(endpoint)
    trend_names = []

    try:
        print(f"Connecting to OPC UA server at {endpoint}...")
        client.connect()
        print("Connected.")

        trend_node = client.get_node(probe_node_id)
        children = trend_node.get_children()

        print(f"Found {len(children)} children in Probe1.Trends:")
        for child in children:
            try:
                name = child.get_display_name().Text
                print(f" - {name}")
                trend_names.append(name)
            except Exception as e:
                print(f"Could not get name for node {child}: {e}")

    except Exception as e:
        print(f"Error: {e}")
    finally:
        client.disconnect()
        print("Disconnected.")

    return trend_names


endpoint = "opc.tcp://localhost:62552/iCOpcUaServer"
trend_names = extract_trend_node_names(endpoint)

Connecting to OPC UA server at opc.tcp://localhost:62552/iCOpcUaServer...
Connected.
Found 7 children in Probe1.Trends:
 - Probe Temp
 - cage
 - Peak at 1155 cm-1
 - Peak at 1680 cm-1
 - Peak at 1088 cm-1
 - Peak at 1649 cm-1
 - THF Peak at 1264 cm-1
Disconnected.


In [4]:
import os

logs_dir = "logs"
print(f"Logs directory exists? {os.path.exists(logs_dir)}")
print(f"Is writable? {os.access(logs_dir, os.W_OK)}")

if not os.path.exists(logs_dir):
    print("Creating logs directory...")
    os.makedirs(logs_dir)

if not os.access(logs_dir, os.W_OK):
    print("Logs directory is not writable! Please fix permissions.")
else:
    print("Logs directory is writable.")


Logs directory exists? True
Is writable? True
Logs directory is writable.


In [5]:
import os
import sqlite3

db_path = "ReactIR.db"

print(f"Database file exists? {os.path.exists(db_path)}")
print(f"Database file writable? {os.access(db_path, os.W_OK)}")

conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# List tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print("Tables in DB:", tables)

# Replace 'TrendTableName' below with the actual table your create_new_trend() writes to:
trend_table_name = "trends"  # Change as needed!

if (trend_table_name,) in tables:
    cursor.execute(f"PRAGMA table_info({trend_table_name});")
    columns = cursor.fetchall()
    print(f"Columns in '{trend_table_name}' table:", columns)
else:
    print(f"Table '{trend_table_name}' does not exist in the database.")

conn.close()


Database file exists? True
Database file writable? True
Tables in DB: [('Users',), ('Projects',), ('Experiments',), ('Documents',), ('Probes',), ('Samples',), ('Spectra',), ('Reagents',), ('Trends',), ('sqlite_sequence',), ('ProbeTempSamples',), ('PeakSamples',)]
Table 'trends' does not exist in the database.


In [4]:
from db_utils import setup_database

db_path = "ReactIR.db"
setup_database(db_path)
print("✅ Database schema initialized or verified.")


Error logger has not been configured with a path.
✅ Database schema initialized or verified.


In [1]:
import sqlite3

# Replace this with your actual database file
db_path = 'ReactIR.db'

# Connect to the SQLite database
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Get all table names
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# Print each table's contents
for table_name in tables:
    table_name = table_name[0]
    print(f"\n--- Contents of table: {table_name} ---")
    
    # Get all data from the table
    cursor.execute(f"SELECT * FROM {table_name}")
    rows = cursor.fetchall()

    # Get column names
    column_names = [description[0] for description in cursor.description]
    print(" | ".join(column_names))
    
    for row in rows:
        print(" | ".join(str(cell) for cell in row))

# Clean up
conn.close()



--- Contents of table: Users ---
UserID | Username
1 | FlowIR

--- Contents of table: Projects ---
ProjectID | Name | UserID
1 | Project Alpha | 1

--- Contents of table: Experiments ---
ExperimentID | Name | ProjectID
1 | Exp 001 | 1

--- Contents of table: Documents ---
DocumentID | Name | ExperimentID
1 | Doc 1 | 1
2 | Unknown_Experiment | None
3 | Unknown_Experiment | None
4 | Unknown_Experiment | None
5 | Unknown_Experiment | None
6 | Unknown_Experiment | None
7 | Unknown_Experiment | None
8 | Unknown_Experiment | None

--- Contents of table: Probes ---
ProbeID | Description | DocumentID | LatestTemperatureCelsius | LatestTemperatureTime
1 | Probe A Description | 1 | 23.5 | 2025-07-03 14:00:00
2 | No description | -1 | None | None
3 | No description | -1 | None | None
4 | No description | -1 | None | None
5 | No description | -1 | None | None
6 | No description | -1 | None | None
7 | No description | -1 | None | None
8 | No description | -1 | None | None
9 | No description | -1 |

In [2]:
import sqlite3
import pandas as pd

# Path to your SQLite database
db_path = 'ReactIR.db'

# Connect to database
conn = sqlite3.connect(db_path)

# Get all table names
table_query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql(table_query, conn)

# Print each table as a DataFrame
for table_name in tables['name']:
    print(f"\n--- Table: {table_name} ---")
    df = pd.read_sql(f"SELECT * FROM {table_name}", conn)
    print(df)

# Close connection
conn.close()




--- Table: Users ---
   UserID Username
0       1   FlowIR

--- Table: Projects ---
   ProjectID           Name  UserID
0          1  Project Alpha       1

--- Table: Experiments ---
   ExperimentID     Name  ProjectID
0             1  Exp 001          1

--- Table: Documents ---
    DocumentID                         Name  ExperimentID  \
0            1                        Doc 1           1.0   
1            2           Unknown_Experiment           NaN   
2            3           Unknown_Experiment           NaN   
3            4           Unknown_Experiment           NaN   
4            5           Unknown_Experiment           NaN   
5            6           Unknown_Experiment           NaN   
6            7           Unknown_Experiment           NaN   
7            8           Unknown_Experiment           NaN   
8            9  MON5.2_Clone_test_run_2_3_2           NaN   
9           10  MON5.2_Clone_test_run_2_3_2           NaN   
10          11  MON5.2_Clone_test_run_2_3_2   

In [3]:
import sqlite3

conn = sqlite3.connect("ReactIR.db")
cursor = conn.cursor()

try:
    cursor.execute("ALTER TABLE Documents ADD COLUMN ErrorLogPath TEXT;")
    conn.commit()
    print("Column added successfully.")
except sqlite3.OperationalError as e:
    print(f"Error: {e} (maybe column already exists?)")
finally:
    conn.close()


Column added successfully.


📁 Initial Error log file: logs\startup\error_log_09-08-2025_20-15-37.txt
Connected to OPC UA server at opc.tcp://localhost:62552/iCOpcUaServer (Attempt 1)

⏳ Waiting for experiment to fully initialise...
🔄 Attempt 1/3: Checking Trend node...
✅ Trends node ready with 7 children.

📡 Probe 1 Metadata
--------------------------------------------------
Probe Name                    : ReactIR 700 C229468110:Probe A
Probe Description             : ReactIR 700; SN: C229468110; Detector: TEMCT; Apodization: Norton-Beer Medium; Probe: DiComp (Diamon... (len=230)
Probe Status                  : Running
Document Name                 : MON5.2_Clone_test_run_2_3_4
Experiment Name               : MON5.2_Clone_test_run_2_3_4
Suffix                        : 
User Name                     : 
Project Name                  : 
Sample Count                  : 439
Current Sampling Interval     : 30
Minimum Sampling Interval     : 5
Maximum Sampling Interval     : 3600
Last Sample Time              : 2025-08-09 20:15:37
Last Sample Raw Spectra       : [0.000596593092787985, 0.0008000511876519151, 0.0010524041628334959, 0.0012870656208652325, 0.0013759056935578392] ... [-0.016881212341164152, -0.022706497036352486, 0.008397204896983064, 0.017209772949053584, 0.004621083719636425] (len=839)
Last Sample Background Spectra: [300.72586123160966, 302.759056037904, 304.58993935555134, 304.79953811107265, 305.09512217176587] ... [3.6251063515686193, 3.224730640825703, 3.606525053838185, 2.9799669785191396, 3.0949578195374334] (len=839)
Last Sample Treated Spectra   : [0.000596593092787985, 0.0008000511876519151, 0.0010524041628334959, 0.0012870656208652325, 0.0013759056935578392] ... [-0.016881212341164152, -0.022706497036352486, 0.008397204896983064, 0.017209772949053584, 0.004621083719636425] (len=839)
--------------------------------------------------

📝 Updated Error log path: logs\MON5.2_Clone_test_run_2_3_4\error_log_09-08-2025_20-15-37.txt

📊 Found 7 children in Probe1.Trends

📈 Peaks Detected:
• Probe Temp
• cage
• Peak at 1155 cm-1
• Peak at 1680 cm-1
• Peak at 1088 cm-1
• Peak at 1649 cm-1
• THF Peak at 1264 cm-1
Waiting for probe to start ...

🚀 Starting trend sampling...
🔍 Monitoring probe status...

Sampling started ... Press Ctrl + C to stop.
Probe is now running. Beginning data capture ...

🟢 Probe status changed: Running
Initial spectrum (sample): [0.000596593092787985, 0.0008000511876519151, 0.0010524041628334959, 0.0012870656208652325, 0.0013759056935578392]
Number of points in spectrum: 839
Wavenumber axis (sample): [4000.0, 3996.0, 3992.0, 3988.01, 3984.01]
Logging started. Press Ctrl+C to stop.

Error during sampling: "The attribute is not supported for the specified Node."(BadAttributeIdInvalid)
Read spectrum (sample): [0.000596593092787985, 0.0008000511876519151, 0.0010524041628334959, 0.0012870656208652325, 0.0013759056935578392]
Attempting to write spectrum to: C:\Users\FlowChemIR_User\Desktop\AGS Users\Meg\Automating-inline-IR-analysis-for-optimising-synthetic-reactions-MSc-project-1\logs\MON5.2_Clone_test_run_2_3_4\spectra\spectrum_run_09-08-2025_20-15-37\raw_spectrum_09-08-2025_20-16-07_995.csv
Spectrum CSV written successfully.
🔍 Treated Spectrum Type: <class 'list'>
🔍 Treated Spectrum Preview: [0.000596593092787985, 0.0008000511876519151, 0.0010524041628334959, 0.0012870656208652325, 0.001375]
Probe metadata keys: ['Probe Name', 'Probe Description', 'Probe Status', 'Document Name', 'Experiment Name', 'Suffix', 'User Name', 'Project Name', 'Sample Count', 'Current Sampling Interval', 'Minimum Sampling Interval', 'Maximum Sampling Interval', 'Last 
Sample Time', 'Last Sample Raw Spectra', 'Last Sample Background Spectra', 'Last Sample Treated Spectra']
✅ Treated spectrum saved to logs\MON5.2_Clone_test_run_2_3_4\spectra\spectrum_run_09-08-2025_20-15-37\treated_spectrum_09-08-2025_20-16-07_995.csv
Inserting data into DB...
✅ Inserted 1 spectrum for DocumentID 33.
Spectrum logged to logs\MON5.2_Clone_test_run_2_3_4\spectra\spectrum_run_09-08-2025_20-15-37\raw_spectrum_09-08-2025_20-16-07_995.csv at 20-16-08
Sleeping for 30 seconds...

❗ Logging interrupted by user (Ctrl+C).

🔌 Disconnected from OPC UA server.


In [2]:
import sqlite3

# Replace this with your actual database file
db_path = 'ReactIR.db'

# Connect to the SQLite database
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Get all table names
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# Print each table's contents (headers + last 6 rows if available)
for table_name in tables:
    table_name = table_name[0]
    print(f"\n--- Contents of table: {table_name} ---")
    
    # Get column names
    cursor.execute(f"PRAGMA table_info({table_name})")
    columns_info = cursor.fetchall()
    column_names = [col[1] for col in columns_info]
    print(" | ".join(column_names) if column_names else "[No columns]")

    # Try to fetch last 6 rows
    cursor.execute(f"SELECT * FROM {table_name} ORDER BY rowid DESC LIMIT 6")
    rows = cursor.fetchall()

    if rows:
        # Reverse so rows print in correct order
        for row in rows[::-1]:
            print(" | ".join(str(cell) if cell is not None else "NULL" for cell in row))
    else:
        print("[No rows]")

# Clean up
conn.close()



--- Contents of table: Users ---
UserID | Username
1 | FlowIR

--- Contents of table: Projects ---
ProjectID | Name | UserID
1 | Project Alpha | 1

--- Contents of table: Experiments ---
ExperimentID | Name | ProjectID
1 | Exp 001 | 1

--- Contents of table: Documents ---
DocumentID | Name | ExperimentID | ErrorLogPath
35 | MON5.2_Clone_test_run_2_3_4 | NULL | logs\MON5.2_Clone_test_run_2_3_4\error_log_09-08-2025_22-18-28.txt
36 | MON5.2_Clone_test_run_2_3_4 | NULL | logs\MON5.2_Clone_test_run_2_3_4\error_log_09-08-2025_22-21-30.txt
37 | MON5.2_Clone_test_run_2_3_5 | NULL | logs\MON5.2_Clone_test_run_2_3_5\error_log_14-08-2025_13-10-07.txt
38 | MON5.2_Clone_test_run_2_3_6 | NULL | logs\MON5.2_Clone_test_run_2_3_6\error_log_18-08-2025_11-07-04.txt
39 | MON5.2_Clone_test_run_2_3_6 | NULL | logs\MON5.2_Clone_test_run_2_3_6\error_log_18-08-2025_11-09-19.txt
40 | AP-ISO-FLOW-11-A | NULL | logs\AP-ISO-FLOW-11-A\error_log_18-08-2025_12-52-35.txt

--- Contents of table: Probes ---
ProbeID | D

In [1]:
import sqlite3

# Replace this with your actual database file
db_path = 'ReactIR.db'

# Connect to the SQLite database
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# === USER INPUTS ===
table_name = "Experiments"       # change this to the correct table
id_column = "experiment_id"      # change this to the actual identifier column
experiment_id = 42               # change this to the experiment you want

# Query for the specific experiment
query = f"SELECT * FROM {table_name} WHERE {id_column} = ?"
cursor.execute(query, (experiment_id,))
rows = cursor.fetchall()

# Get column names for pretty printing
column_names = [description[0] for description in cursor.description]

print(f"\n--- Experiment {experiment_id} from table {table_name} ---")
if rows:
    print(" | ".join(column_names))
    for row in rows:
        print(" | ".join(str(cell) if cell is not None else "NULL" for cell in row))
else:
    print("No matching experiment found.")

# Clean up
conn.close()


OperationalError: no such column: experiment_id

In [ ]:
import sqlite3

db_path = 'ReactIR.db'
search_value = 40  # The experiment ID that is being searched for

conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Get all table names
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

print(f"Searching for value {search_value} in all tables...\n")

for table_name in tables:
    table_name = table_name[0]

    # Get column names for the table
    cursor.execute(f"PRAGMA table_info({table_name})")
    columns_info = cursor.fetchall()
    column_names = [col[1] for col in columns_info]

    if not column_names:
        continue

    for col in column_names:
        try:
            query = f"SELECT * FROM {table_name} WHERE {col} = ?"
            cursor.execute(query, (search_value,))
            rows = cursor.fetchall()

            if rows:
                print(f"\n--- Match in table '{table_name}', column '{col}' ---")
                print(" | ".join(column_names))
                for row in rows:
                    print(" | ".join(str(cell) if cell is not None else "NULL" for cell in row))
        except sqlite3.OperationalError:
            # Skip if column type doesn't support '=' comparison
            continue

conn.close()


Searching for value 40 in all tables...


--- Match in table 'Documents', column 'DocumentID' ---
DocumentID | Name | ExperimentID | ErrorLogPath
40 | AP-ISO-FLOW-11-A | NULL | logs\AP-ISO-FLOW-11-A\error_log_18-08-2025_12-52-35.txt

--- Match in table 'Probes', column 'ProbeID' ---
ProbeID | Description | DocumentID | LatestTemperatureCelsius | LatestTemperatureTime
40 | No description | -1 | NULL | NULL

--- Match in table 'Probes', column 'DocumentID' ---
ProbeID | Description | DocumentID | LatestTemperatureCelsius | LatestTemperatureTime
455 | ReactIR 700; SN: C229468110; Detector: TEMCT; Apodization: Norton-Beer Medium; Probe: DiComp (Diamond); SN: C226337177 ; Interface: DS Micro Flow Cell; Sampling: 4000 to 650 cm-1; Resolution: 8; Scan option: AutoSelect; Gain: Low; | 40 | NULL | NULL
456 | ReactIR 700; SN: C229468110; Detector: TEMCT; Apodization: Norton-Beer Medium; Probe: DiComp (Diamond); SN: C226337177 ; Interface: DS Micro Flow Cell; Sampling: 4000 to 650 cm-1; Resolutio